In [ ]:
pip install numpy-stl

In [ ]:
# STL bounding box
# pip install numpy-stl
import numpy as np
from stl import mesh
import gzip
import os

def is_compressed(file_path):
    return file_path.endswith('.gz')

def decompress_stl_gz(gz_file_path, decompressed_file_path):
    with gzip.open(gz_file_path, 'rb') as f_in:
        with open(decompressed_file_path, 'wb') as f_out:
            f_out.write(f_in.read())

def get_bounding_box(stl_file_path):
    # Load the STL file
    your_mesh = mesh.Mesh.from_file(stl_file_path)

    # Get the minimum and maximum coordinates along each axis
    min_x, max_x = np.min(your_mesh.x), np.max(your_mesh.x)
    min_y, max_y = np.min(your_mesh.y), np.max(your_mesh.y)
    min_z, max_z = np.min(your_mesh.z), np.max(your_mesh.z)

    # Define the bounding box
    bounding_box = {
        'hullXmin': min_x, 'hullXmax': max_x,
        'hullYmin': min_y, 'hullYmax': 0,
        'hullZmin': min_z, 'hullZmax': max_z
    }

    return bounding_box

def write_bounding_box_to_file(bounding_box, file_path):
    with open(file_path, 'w') as file:
        for key, value in bounding_box.items():
            file.write(f"{key}  {value:.4f};\n")

# Example usage
file_path = 'hullDTC.stl'
decompressed_file_path = 'decompressed_file.stl'
output_file_path = 'bounding_box.txt'

if is_compressed(file_path):
    # Decompress the STL file if it is compressed
    decompress_stl_gz(file_path, decompressed_file_path)
    stl_file_path = decompressed_file_path
else:
    stl_file_path = file_path

# Get the bounding box of the STL file
bounding_box = get_bounding_box(stl_file_path)

# Write the bounding box to a file
write_bounding_box_to_file(bounding_box, output_file_path)

# Remove the decompressed file if it was created
if is_compressed(file_path):
    os.remove(decompressed_file_path)

print(f"Bounding box data written to {output_file_path}")


In [ ]:
import math
import gmsh

igs_file = 'dtc.igs'
scale = 0.001 * 1/60
# Initialize gmsh
gmsh.initialize()

# Merge the IGS file: This loads the IGS file into Gmsh.
# gmsh.merge(igs_file)

# Classify Surfaces: This step is necessary to prepare the surfaces for meshing.
angle = 20  # Angle in degrees, below which two surfaces are considered to be on the same plane
forceParametrizablePatches = True
includeBoundary = True
curveAngle = 180

# gmsh.model.mesh.classifySurfaces(angle * math.pi / 180., includeBoundary, forceParametrizablePatches, curveAngle * math.pi / 180.)

# Create a volume from the classified surfaces
# gmsh.model.mesh.createGeometry()

# Set the meshing algorithm
# 2D mesh algorithm (1: MeshAdapt, 2: Automatic, 3: Initial mesh only, 5: Delaunay, 6: Frontal-Delaunay, 7: BAMG, 8: Frontal-Delaunay for Quads, 9: Packing of Parallelograms, 11: Quasi-structured Quad)
# Default value: 6
# gmsh.option.setNumber("Mesh.Algorithm", 1)  

# Optionally, set some options for the meshing process
# For example, setting the mesh size:
gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 100)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 1000)

# Mesh the geometry
# gmsh.model.mesh.generate(3)

# # Optimize the mesh
# gmsh.model.mesh.optimize("HighOrder") # Netgen, HighOrder

# # Synchronize the internal CAD representation with the Gmsh model
# gmsh.model.occ.synchronize()

# Apply scaling transformation
gmsh.model.occ.dilate(gmsh.model.getEntities(), 0, 0, 0, scale, scale, scale)

# Synchronize the internal CAD representation with the Gmsh model
gmsh.model.occ.synchronize()

# Save mesh to STL file (replace 'output_file.stl' with your desired output file path)
# gmsh.write('hull_surface.stl')

# Finalize gmsh
gmsh.finalize()

In [ ]:
import bpy

# Open a new blank file (without default cube and camera)
bpy.ops.wm.read_factory_settings(use_empty=True)

# Ensure the 3D Print Toolbox is enabled (for making manifold)
if not bpy.context.preferences.addons.get('object_print3d_utils'):
    bpy.ops.preferences.addon_enable(module='object_print3d_utils')

# Import the STL file
bpy.ops.import_mesh.stl(filepath="hull_surface.stl")
# bpy.ops.import_mesh.stl(filepath="kcs_bothHalves.stl")

### Make manifold and calculate volume ###
# Ensure we have the object selected
obj = bpy.context.selected_objects[0]

# Make Manifold using 3D Print Toolbox
bpy.context.view_layer.objects.active = obj
bpy.ops.object.mode_set(mode='EDIT')
bpy.ops.mesh.print3d_clean_non_manifold()
bpy.ops.object.mode_set(mode='OBJECT')

# Save the STL file with the manifold changes
bpy.ops.export_mesh.stl(filepath="hull_solid.stl", check_existing=False, use_selection=True)
print("Manifold hull saved to hull_solid.stl")

In [ ]:
import gmsh
import sys

def convert_igs_to_stl(input_file, needMirror, output_file, quality_threshold):
    gmsh.initialize()

    try:
        gmsh.model.add("model")

        # Import IGES file
        gmsh.model.occ.importShapes(input_file)
        gmsh.model.occ.synchronize()

        # Mirror the mesh at the XY plane
        if needMirror:
            orig = gmsh.model.getEntities()
            copy = gmsh.model.occ.copy(orig)
            gmsh.model.occ.mirror(copy, 0,1,0,0) # Normal vector of the mirror plane and a point on the mirror plane
            # gmsh.model.occ.fuse(orig, copy) # didn't work
            gmsh.model.occ.synchronize()

        # Scale the mesh
        scale = 0.001 * 1/60
        gmsh.model.occ.dilate(gmsh.model.getEntities(), 0, 0, 0, scale, scale, scale)
        gmsh.model.occ.synchronize()

        # Get the bounding box of the imported geometry, covering all surfaces
        # Get all surfaces in the model
        surface_tags = gmsh.model.getEntities(dim=2)
        # Initialize bounding box extents
        minX = minY = minZ = float('inf')
        maxX = maxY = maxZ = float('-inf')

        # Loop through all surfaces to find the overall bounding box
        for surface in surface_tags:
            tag = surface[1]
            bbox = gmsh.model.getBoundingBox(2, tag)
            minX = min(minX, bbox[0])
            minY = min(minY, bbox[1])
            minZ = min(minZ, bbox[2])
            maxX = max(maxX, bbox[3])
            maxY = max(maxY, bbox[4])
            maxZ = max(maxZ, bbox[5])

        # Print the overall bounding box
        print(f"Overall Bounding Box: minX={minX}, minY={minY}, minZ={minZ}, maxX={maxX}, maxY={maxY}, maxZ={maxZ}")

        eleLengthMax = (maxX - minX)/350

        # Set meshing parameters
        gmsh.option.setNumber("Mesh.CharacteristicLengthMin", eleLengthMax*0.01)
        gmsh.option.setNumber("Mesh.CharacteristicLengthMax", eleLengthMax)

        # Set the meshing algorithm
        # gmsh.option.setNumber("Mesh.Algorithm", 5)

        # Generate 3D mesh
        gmsh.model.mesh.generate()

        # # Stich the surfaces together
        # # Stitch the surfaces together (Create a surface loop and volume)
        # surface_loop = gmsh.model.occ.addSurfaceLoop([s[1] for s in surface_tags])
        # gmsh.model.occ.addVolume([surface_loop])
        # gmsh.model.occ.synchronize()


        # Optimize the mesh if quality threshold is specified
        if quality_threshold > 0:
            gmsh.option.setNumber("Mesh.OptimizeThreshold", quality_threshold)
            gmsh.model.mesh.optimize("Netgen") # Netgen, HighOrder
            print(f"Mesh optimized with quality threshold: {quality_threshold}")

        # Scaling here causing mesh to change, uniformaty)

        # # Mirror the mesh at the XY plane
        # orig = gmsh.model.getEntities()
        # copy = gmsh.model.occ.copy(orig)
        # gmsh.model.occ.mirror(copy, 0,0,1,0)
        # # gmsh.model.occ.fuse(orig, copy)
        # gmsh.model.occ.synchronize()


        # Save mesh to STL
        gmsh.write(output_file)

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        gmsh.finalize()


# Input and output file paths
input_igs = "kcs.igs"
output_stl = "hull_surface.stl"
needMirror = True

# Mesh options
quality_threshold = 0  # Set to 0 to disable optimization

convert_igs_to_stl(input_igs, needMirror, output_stl, quality_threshold)

In [4]:
pip install salome

Note: you may need to restart the kernel to use updated packages.


  ERROR: Could not find a version that satisfies the requirement salome (from versions: none)
ERROR: No matching distribution found for salome
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
import salome
salome.salome_init_without_session()
import GEOM
from salome.geom import geomBuilder
geompy = geomBuilder.New()
 
import tempfile, os
 
# create a sphere
sphere = geompy.MakeSphereR(100)
 
tmpdir = tempfile.mkdtemp()

# export sphere to the ASCII STL file, with custom deflection coefficient
f_stl2 = os.path.join(tmpdir, "sphere2.stl")
geompy.ExportSTL(sphere, f_stl2, True, 0.1)

ModuleNotFoundError: No module named 'salome'